In [1]:
from dataservice import DataService
from IPython.display import Markdown, display
import seaborn as sns 
import matplotlib.pyplot as plt 
import pandas as pd 
from datetime import * 
import re
import plotly.express as px 

In [2]:
obj = DataService()
obj.set_source("https://my.raceresult.com/309137/")
obj.handle_cookie_banner() 
obj.click_show_all_buttons() 
obj.create_soup() 
obj.get_metadata() 
obj.expand_table() 
obj.get_table_as_df() 
data = obj.dataFrame

[DEBUG] DataService initialized.
[DEBUG] Navigated to URL: https://my.raceresult.com/309137/
[DEBUG] Source set to: https://my.raceresult.com/309137/
[DEBUG] Current source: https://my.raceresult.com/309137/
Cookie-Banner geschlossen.


In [6]:
print(data.dtypes)


Pl.       object
Snr.      object
Name      object
Nat.      object
Jg.       object
Verein    object
AK-Pl.    object
MW-Pl.    object
Zeit      object
dtype: object


In [7]:
obj.set_time()

In [8]:
print(obj.dataFrame.head(10))

   Pl.  Snr.                         Name Nat.   Jg.                   Verein  \
0   1.  1383             BECKER Jan Lukas       1993             Sport Jochum   
1   2.  1481              GROßKOPF Julian       2000          LAZ Ludwigsburg   
2   3.  1109              ZEHFELD Lennart       2005             LC Rehlingen   
3   4.  1060  TESF AMARIAM ESTOPIA Selama       1995        TV Alzey Laufteam   
4   5.  1253                BAUER Clemens       2006  TV Waldstraße Wiesbaden   
5   6.   783                 FINK Leander       2001        TV Alzey Laufteam   
6   7.   373                  LÖSEL Arvid       2004      LG Region Karlsruhe   
7   8.   777                    FAST Emil       1998                            
8   9.  1453                  HILLE Benno       2001              LSF Münster   
9  10.  1062              TRAMPUSCH Simon       2003          TuS Framersheim   

      AK-Pl. MW-Pl.      Zeit  Zeit_minutes  
0     1. M30   1. M  00:29:25     29.416667  
1       1. M   2

In [3]:
race = DataService()

df = race.scrape_data("https://my.raceresult.com/309137/")

[DEBUG] DataService initialized.
[DEBUG] Navigated to URL: https://my.raceresult.com/309137/
[DEBUG] Source set to: https://my.raceresult.com/309137/
[DEBUG] Current source: https://my.raceresult.com/309137/
Cookie-Banner geschlossen.


/Users/janikwahrheit/Library/CloudStorage/OneDrive-Persönlich/05_Code/Python/Webscraping_Race/dataservice.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.dataFrame["datetime"] = pd.to_datetime(self.dataFrame["datetime"])


In [4]:
df.head()

,Pl.,Snr.,Name,Jg.,Verein,AK-Pl.,MW-Pl.,Zeit,datetime,AK,M/W
0,1.,1383,BECKER Jan Lukas,1993,Sport Jochum,1. M30,1. M,00:29:25,2025-01-06 00:29:25,M30,M
1,2.,1481,GROßKOPF Julian,2000,LAZ Ludwigsburg,1. M,2. M,00:30:21,2025-01-06 00:30:21,M,M
2,3.,1109,ZEHFELD Lennart,2005,LC Rehlingen,2. M,3. M,00:30:41,2025-01-06 00:30:41,M,M
3,4.,1060,TESF AMARIAM ESTOPIA Selama,1995,TV Alzey Laufteam,2. M30,4. M,00:30:47,2025-01-06 00:30:47,M30,M
4,5.,1253,BAUER Clemens,2006,TV Waldstraße Wiesbaden,1. MJ U20,5. M,00:30:57,2025-01-06 00:30:57,MJ U20,M


In [6]:
import plotly.express as px
import datetime as dt

#for index, time in enumerate(df["Zeit"]): 
#    df["Zeit"].iloc[index] = dt.datetime.combine(dt.date(2025,1,1), time)

# Create the histogram
fig = px.histogram(
    df, 
    x="datetime", 
    nbins=200, 
    title="Verteilung der Rennzeiten",  # Add a title
    color_discrete_sequence=["#b81212"] # Custom color
)

# Customize layout
fig.update_layout(
    title_font=dict(size=20, family="Arial", color="black"),  # Title font style
    xaxis_title = "Zeit in Minuten", 
    yaxis_title = "Anzahl",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),  # Add gridlines
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    plot_bgcolor="white",  # Background color
    bargap=0.2,  # Gap between bars
    xaxis_tickformat = "%H:%M:%S"
)


# Customize bars
fig.update_traces(
    marker=dict(
        line=dict(width=1, color="black")  # Add border to bars
    ),
    opacity=0.8, 
    hovertemplate="<b>Zeit in Minuten:</b> %{x}<br>" +  # Customize hover text
              "<b>Anzahl:</b> %{y}<extra></extra>" 
)

# Show the plot
fig.show()


In [7]:
boxplot_race_time = px.box(
  df, 
  y = "Zeit", 
  title = "Boxplot der Rennzeiten", 
  color_discrete_sequence=["#b81212"]  
)

# Customize layout
boxplot_race_time.update_layout(
    title_font=dict(size=20, family="Arial", color="black"), 
    yaxis_title="Zeit in Minuten",  # Y-axis title
    xaxis=dict(showgrid=True, gridcolor="lightgray"),  
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    plot_bgcolor="white",  
)


boxplot_race_time.show() 

In [8]:
import plotly.express as px

# Assuming 'data' is your DataFrame
# Count the occurrences of 'M' and 'W' in the "M/W" column
gender_counts = df['M/W'].value_counts()


# Assuming 'gender_counts' is already calculated (counts of 'M' and 'W')
fig = px.pie(
    names=gender_counts.index,  # The categories (M and W)
    values=gender_counts.values,  # The count of each category
    title="Proportion of Male/Female Participants",  # Title of the pie chart
    color=gender_counts.index,  # Color by gender categories (M and W)
    color_discrete_map={"M": "#b81212", "W": "#ffffff"}
)

# Add a red border around the white sector (Female category)
fig.update_traces(
     hovertemplate="<b>Geschlecht: </b> %{label}<br>" +  # Display M/W category
                  "<b>Teilnehmer:</b> %{value} <extra></extra>",  # Display the count value
    marker=dict(
        line=dict(
            color="#b81212",  # Red color for the border
            width=1  # Set the width of the border
        )
    )
)

# Show the pie chart
fig.show()




In [16]:
# Create the scatter plot with a regression line
fig = px.scatter(
    df, 
    y="datetime",  # Time on the x-axis
    x="AK",  # Age class on the y-axis
    title="Relation Between Age Class and Race Time",  # Add a title
    color_discrete_sequence=["#b81212"],  # Custom color
)

# Customize layout
fig.update_layout(
    title_font=dict(size=20, family="Arial", color="black"),  # Title font style
    yaxis_title="Zeit (Race Time)",  # X-axis label
    xaxis_title="Altersklasse (AK)",  # Y-axis label
    yaxis=dict(
        showgrid=True, gridcolor="lightgray", tickformat="%H:%M:%S"  # Format time
    ),
    xaxis=dict(showgrid=True, gridcolor="lightgray"),  # Gridlines for y-axis
    plot_bgcolor="white",  # Background color
)

fig.show()


In [20]:
# Extract the numerical value from the "AK" column and use it for sorting
df['AK_numeric'] = df['AK'].str.extract(r'(\d+)$').fillna(0).astype(int)

# Sort the dataframe by the numeric value in "AK_numeric"
df_scatter = df.sort_values(by='AK_numeric')

# Create the scatter plot with sorted x-axis and "M/W" as hue
fig = px.scatter(
    df_scatter,
    y="datetime",  # Race time
    x="AK",  # Age class
    color="M/W",  # Hue for gender
    title="Relation Between Age Class and Race Time by Gender",
    color_discrete_map={"M": "#b81212", "W": "#000000"},  # Custom colors for M/W
)

# Customize layout
fig.update_layout(
    title_font=dict(size=20, family="Arial", color="black"),  # Title font style
    yaxis_title="Zeit (Race Time)",  # Y-axis label
    xaxis_title="Altersklasse (AK)",  # X-axis label
    yaxis=dict(
        showgrid=True, gridcolor="lightgray", tickformat="%H:%M:%S"  # Format time
    ),
    xaxis=dict(
        showgrid=True, gridcolor="lightgray",  # Gridlines for x-axis
        categoryorder="array",  # Order x-axis categories manually
        categoryarray=df['AK']  # Use sorted "AK" values
    ),
    plot_bgcolor="white",  # Background color
)

fig.show()